In [1]:
import os
import nltk
import sklearn
import random

In [3]:
files_pos = os.listdir('train/pos')
files_pos = [open('train/pos/'+f, 'r').read() for f in files_pos]
files_neg = os.listdir('train/neg')
files_neg = [open('train/neg/'+f, 'r').read() for f in files_neg]


'pos'

In [14]:
from nltk import word_tokenize, PorterStemmer, SnowballStemmer
from nltk.stem import *
import re

def remove_punctuation_digits(sentence): 
    sentence = re.sub(r'[^(a-zA-Z)\s]','', sentence)
    return sentence 


from nltk.corpus import stopwords
stop_words = list(set(stopwords.words('english')))
def remove_stopword(sentence):
    return [w for w in sentence if not w in stop_words]

stemmer = SnowballStemmer("english")
def clean_reviews(list_rev): 
    listy = []
    for rev in list_rev: 
        cleaned_review = re.sub(r'[^(a-zA-Z)\s]','', rev)
        tokenized = word_tokenize(cleaned_review)
        stopped = remove_stopword(tokenized)
        stemmed = [stemmer.stem(w) for w in stopped]
        listy.append(stemmed)
    return listy
        

def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features
def make_tuple(document, sent): 
    listy = []
    for review in document: 
        listy.append((review, sent))
    return listy

In [5]:
cleaned_pos = clean_reviews(files_pos)
cleaned_neg = clean_reviews(files_neg)
cleaned_all = cleaned_pos+cleaned_neg
random.shuffle(cleaned_all)

In [6]:
tagged_pos = make_tuple(cleaned_pos, 'pos')
tagged_neg = make_tuple(cleaned_neg, 'neg')
tagged_all = tagged_pos + tagged_neg
random.shuffle(tagged_all)

In [ ]:
tagged_all2 = 

In [7]:
all_words = [w.lower() for review in cleaned_all for w in review]

In [10]:

all_words2 = nltk.FreqDist(all_words)

word_features = list(all_words2.keys())[:3000]


In [8]:
print(len(cleaned_pos), len(cleaned_neg))

12500 12500


In [12]:
featuresets_pos = [(find_features(rev), category) for (rev, category) in tagged_pos]
featuresets_neg = [(find_features(rev), category) for (rev, category) in tagged_neg]

In [139]:
test_pos = featuresets_pos[:100]
test_neg = featuresets_neg[:100]

In [22]:


featuresets = [(find_features(rev), category) for (rev, category) in tagged_all ]
training_set = featuresets[:20000]
testing_set = featuresets[20000:]

In [23]:
print(featuresets[0])

({'humm': False, 'italian': False, 'movi': True, 'star': False, 'david': False, 'hasselhoff': False, 'linda': False, 'blair': False, 'i': True, 'wasnt': False, 'expect': False, 'much': False, 'honest': False, 'fact': False, 'took': False, 'even': False, 'less': False, 'it': True, 'doesnt': False, 'mean': False, 'worst': False, 'seen': True, 'watch': False, 'wors': False, 'thing': False, 'plot': False, 'time': False, 'confus': False, 'uninterest': False, 'good': False, 'gore': False, 'scene': False, 'save': False, 'apart': False, 'go': False, 'love': False, 'special': False, 'effect': False, 'realli': False, 'cheesi': False, 'bad': False, 'now': False, 'want': False, 'troll': False, 'director': False, 'sure': False, 'a': True, 'super': False, 'unusu': False, 'film': True, 'audiard': False, 'read': False, 'my': False, 'lip': False, 'pulpi': False, 'lone': False, 'heart': False, 'thriller': False, 'perfect': False, 'handsom': False, 'grizzl': False, 'charisma': False, 'vincent': False, 'c

In [114]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [115]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 84.28


In [99]:
classifier.show_most_informative_features(15)

Most Informative Features
                semblanc = True              neg : pos    =     25.8 : 1.0
                 unwatch = True              neg : pos    =     25.7 : 1.0
                    mstk = True              neg : pos    =     25.4 : 1.0
                rediscov = True              pos : neg    =     14.3 : 1.0
                flawless = True              pos : neg    =     11.2 : 1.0
               pointless = True              neg : pos    =     10.8 : 1.0
                    gedd = True              pos : neg    =     10.3 : 1.0
                 superbl = True              pos : neg    =     10.1 : 1.0
                    wast = True              neg : pos    =      9.3 : 1.0
                   worst = True              neg : pos    =      9.0 : 1.0
                awardwin = True              pos : neg    =      9.0 : 1.0
                  demill = True              pos : neg    =      8.6 : 1.0
                 olympia = True              pos : neg    =      8.2 : 1.0

In [117]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

Classifiers for an ensemble model: 
Naive Bayes (NB)
Multinomial NB
Bernoulli NB
Logistic Regression
Stochastic Gradient Descent Classifier - SGD
Support Vector Classification - SVC
Linear SVC
Nu- NuSVC

In [118]:
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_clf = SklearnClassifier(MultinomialNB())
MNB_clf.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_clf, testing_set))*100)

BNB_clf = SklearnClassifier(BernoulliNB())
BNB_clf.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BNB_clf, testing_set))*100)

LogReg_clf = SklearnClassifier(LogisticRegression())
LogReg_clf.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogReg_clf, testing_set))*100)

SGD_clf = SklearnClassifier(SGDClassifier())
SGD_clf.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGD_clf, testing_set))*100)

SVC_clf = SklearnClassifier(SVC())
SVC_clf.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_clf, testing_set))*100)

LinSVC_clf = SklearnClassifier(LinearSVC())
LinSVC_clf.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinSVC_clf, testing_set))*100)

NuSVC_clf = SklearnClassifier(NuSVC())
NuSVC_clf.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_clf, testing_set))*100)

Original Naive Bayes Algo accuracy percent: 84.28
Most Informative Features
                semblanc = True              neg : pos    =     25.8 : 1.0
                 unwatch = True              neg : pos    =     25.7 : 1.0
                    mstk = True              neg : pos    =     25.4 : 1.0
                rediscov = True              pos : neg    =     14.3 : 1.0
                flawless = True              pos : neg    =     11.2 : 1.0
               pointless = True              neg : pos    =     10.8 : 1.0
                    gedd = True              pos : neg    =     10.3 : 1.0
                 superbl = True              pos : neg    =     10.1 : 1.0
                    wast = True              neg : pos    =      9.3 : 1.0
                   worst = True              neg : pos    =      9.0 : 1.0
                awardwin = True              pos : neg    =      9.0 : 1.0
                  demill = True              pos : neg    =      8.6 : 1.0
                 olympia

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 83.66
SVC_classifier accuracy percent: 85.0
LinearSVC_classifier accuracy percent: 84.68
NuSVC_classifier accuracy percent: 86.68


In [140]:
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, test_pos))*100)
classifier.show_most_informative_features(15)

print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, test_neg))*100)
classifier.show_most_informative_features(15)

Original Naive Bayes Algo accuracy percent: 80.0
Most Informative Features
                semblanc = True              neg : pos    =     25.8 : 1.0
                 unwatch = True              neg : pos    =     25.7 : 1.0
                    mstk = True              neg : pos    =     25.4 : 1.0
                rediscov = True              pos : neg    =     14.3 : 1.0
                flawless = True              pos : neg    =     11.2 : 1.0
               pointless = True              neg : pos    =     10.8 : 1.0
                    gedd = True              pos : neg    =     10.3 : 1.0
                 superbl = True              pos : neg    =     10.1 : 1.0
                    wast = True              neg : pos    =      9.3 : 1.0
                   worst = True              neg : pos    =      9.0 : 1.0
                awardwin = True              pos : neg    =      9.0 : 1.0
                  demill = True              pos : neg    =      8.6 : 1.0
                 olympia 

In [120]:
classifiers_dict = {'MNB': [MNB_clf, 'MNB_classifier.pickle'],
                    'BNB': [BNB_clf, 'BNB_clf.pickle'],
                    'LogReg': [LogReg_clf, 'LogReg_clf.pickle'],
                    'SGD': [SGD_clf, 'SGD_clf.pickle'], 
                    'SVC': [SVC_clf, 'SVC_clf.pickle'], 
                    'LinSVC': [LinSVC_clf, 'LinSVC_clf.pickle'], 
                    'NuSVC': [NuSVC_clf, 'NuSVC_clf.pickle']}

In [16]:
def create_pickle(classifer, file_name): 
    save_classifier = open(file_name, 'wb')
    pickle.dump(classifier, save_classifier)
    save_classifier.close()
def get_clf(classifer, file_name): 
    classifier_f = open(file_name, "rb")
    classifier = pickle.load(classifier_f)
    classifier_f.close()
    return classifier

In [123]:
for clf, listy in classifiers_dict.items(): 
    create_pickle(listy[0], listy[1])

In [20]:
import pickle
classifier_f = open('MNB_classifier.pickle', "rb")
MNB_clf_picked = pickle.load(classifier_f)
classifier_f.close()

In [24]:
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_clf_picked, testing_set))*100)

MNB_classifier accuracy percent: 84.08


In [26]:
import pickle
classifier_f = open('BNB_clf.pickle', "rb")
BNB_clf_picked = pickle.load(classifier_f)
classifier_f.close()

In [27]:
print("BNB_classifier accuracy percent:", (nltk.classify.accuracy(BNB_clf_picked, testing_set))*100)

BNB_classifier accuracy percent: 84.08


In [29]:
import pickle
classifier_f = open('LogReg_clf.pickle', "rb")
LogReg_clf_picked = pickle.load(classifier_f)
classifier_f.close()

In [30]:
print("BNB_classifier accuracy percent:", (nltk.classify.accuracy(LogReg_clf_picked, testing_set))*100)

BNB_classifier accuracy percent: 84.08


# EXTRA 

In [30]:

from nltk.corpus import movie_reviews

documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

In [34]:

random.shuffle(documents)

all_words = []

for w in movie_reviews.words():
    all_words.append(w.lower())

all_words2 = nltk.FreqDist(all_words)

word_features = list(all_words2.keys())[:3000]


In [83]:
all_words2

FreqDist({',': 77717, 'the': 76529, '.': 65876, 'a': 38106, 'and': 35576, 'of': 34123, 'to': 31937, "'": 30585, 'is': 25195, 'in': 21822, ...})

In [ ]:
# Write a list of ways that you can do feature selection when creating a model.
# remove on of highly correlated variables 
# remove variables that are not necessarily explaining a lot about the model 
# variables that are 
# Write a list of reasons why feature selection is a necessary in the machine learning project process


In [102]:
import pickle 

save_classifier = open('NB_clf_nltk.pickle', 'wb')
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [104]:
classifier_f = open("NB_clf_nltk.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

In [106]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 84.28


In [128]:
from nltk.classify import ClassifierI
from statistics import mode

In [134]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [ ]:
{'MNB': [MNB_clf, 'MNB_classifier.pickle'],
                    'BNB': [BNB_clf, 'BNB_clf.pickle'],
                    'LogReg': [LogReg_clf, 'LogReg_clf.pickle'],
                    'SGD': [SGD_clf, 'SGD_clf.pickle'], 
                    'SVC': [SVC_clf, 'SVC_clf.pickle'], 
                    'LinSVC': [LinSVC_clf, 'LinSVC_clf.pickle'], 
                    'NuSVC': [NuSVC_clf, 'NuSVC_clf.pickle']}

In [133]:
voted_classifier = VoteClassifier(classifier,
                                  NuSVC_clf,
                                  LinSVC_clf,
                                  SGD_clf,
                                  MNB_clf,
                                  BNB_clf,
                                  LogReg_clf)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)

voted_classifier accuracy percent: 86.44
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0
Classification: pos Confidence %: 100.0
Classification: neg Confidence %: 100.0


In [136]:
testing_set[0][1]

'neg'

In [38]:
listy = []
for rev, tag in featuresets: 
    rev['category'] = tag
    listy.append(rev)
    

In [62]:
df = pd.DataFrame.from_dict(listy)
df = df.astype('category')

In [73]:
y = df['category']
df = df.drop(['category'],  axis=1)

In [89]:
X = pd.get_dummies(df, drop_first=True)

In [90]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y)